In [1]:
import pandas as pd 
from pathlib import Path
import os
import numpy as np

In [2]:
cd ..

c:\Gan\GAN-market-prediction


In [3]:
stock_path = Path(os.path.abspath("")) / "data" / "stock_data"
save_path = Path(os.path.abspath("")) / "paper"
tickers = ["EA", "UBSFY", "TTWO", "ATVI"]

In [4]:
df = pd.DataFrame()
for ticker in tickers:
    tmp = pd.read_csv(stock_path / f"{ticker}.csv").tail(178)
    tmp["Company"] = ticker
    df = df.append(tmp)

C:\Users\kuba1\AppData\Local\Temp\ipykernel_9000\748348373.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
C:\Users\kuba1\AppData\Local\Temp\ipykernel_9000\748348373.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
C:\Users\kuba1\AppData\Local\Temp\ipykernel_9000\748348373.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
C:\Users\kuba1\AppData\Local\Temp\ipykernel_9000\748348373.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)


In [5]:
df.groupby(by="Company")

In [6]:
pd.options.display.float_format = '{:.5f}'.format

In [7]:
(
    df.loc[:, [col for col in df.columns if col not in ["Dividends", "Stock Splits"]]]
    .groupby(by="Company")
    .describe()
    .T
)


Company                ATVI            EA          TTWO         UBSFY
Open   count      178.00000     178.00000     178.00000     178.00000
       mean        83.88748     139.35839     170.97678      12.85488
       std         11.44004       5.31964      10.63068       1.80794
       min         56.97780     120.48962     145.91000       9.05000
       25%         77.60000     137.52875     163.15500      11.16750
       50%         83.51000     140.79250     171.31500      12.96005
       75%         93.58500     142.89994     178.88501      14.21000
       max         98.86000     148.54312     192.25999      16.09960
High   count      178.00000     178.00000     178.00000     178.00000
       mean        84.71179     140.86301     172.94974      12.92609
       std         11.44898       5.27167      10.66488       1.80453
       min         57.44000     123.59343     147.10001       9.27000
       25%         78.37250     139.54444     164.68250      11.25895
       50%         84.46250     142.18925     173.34635      13.10500
       75%         94.79250     144.03059     182.23750      14.28000
       max         99.45900     148.55309     195.82500      16.18000
Low    count      178.00000     178.00000     178.00000     178.00000
       mean        82.76441     137.73099     168.86979      12.75559
       std         11.48914       5.47203      10.36375       1.80319
       min         56.40000     119.91839     144.58099       9.05000
       25%         76.66753     136.24021     160.56500      11.04758
       50%         82.65500     139.20772     169.41000      12.84000
       75%         92.75750     141.24267     176.59000      14.12750
       max         97.61000     145.78012     186.42000      16.05000
Close  count      178.00000     178.00000     178.00000     178.00000
       mean        83.64834     139.18954     170.87719      12.83658
       std         11.46632       5.47264      10.50990       1.80542
       min         57.28000     120.06819     145.25000       9.14000
       25%         77.36500     137.80326     162.89500      11.16250
       50%         83.47000     140.66914     171.28000      12.97500
       75%         93.40250     142.60301     178.92750      14.21743
       max         99.18000     148.17406     192.91000      16.13000
Volume count      178.00000     178.00000     178.00000     178.00000
       mean   8236266.19663 2473996.98315 1276613.75281  169367.50562
       std    5413813.44292 1073436.84836  669500.10641  278989.77157
       min    2596873.00000 1016396.00000  559480.00000   16495.00000
       25%    5105944.50000 1766566.25000  876545.75000   46863.25000
       50%    6667305.00000 2181122.50000 1095232.00000   69654.50000
       75%    9477788.50000 2854991.50000 1422011.25000  148630.00000
       max   43753567.00000 8344344.00000 5935523.00000 1833827.00000

In [44]:
with open(save_path / "desc_stock.tex", "w") as f:
    f.write(
        df.loc[
            :, [col for col in df.columns if col not in ["Dividends", "Stock Splits"]]
        ]
        .groupby(by="Company")
        .describe()
        .T.to_latex(caption="Descriptive statistics for chosen companies")
    )


C:\Users\kuba1\AppData\Local\Temp\ipykernel_10816\838483005.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.loc[


In [8]:
results_dict = {
    "ATVI": [1.3585, 1.8867],
    "EA": [1.3427, 1.8956], 
    "TTWO": [3.5405, 4.5069],
    "UBSFY": [0.2578, 0.3376]
}

In [9]:
errs = pd.DataFrame(results_dict).append(df.groupby(by="Company")['Close'].mean()).rename(index={0: "MAE", 1:"RMSE", "Close": "Close Price Mean"})

C:\Users\kuba1\AppData\Local\Temp\ipykernel_9000\3186659599.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  errs = pd.DataFrame(results_dict).append(df.groupby(by="Company")['Close'].mean()).rename(index={0: "MAE", 1:"RMSE", "Close": "Close Price Mean"})


In [18]:
errs.index

Index(['MAE', 'RMSE', 'Close Price Mean'], dtype='object')

In [20]:
errs.index = [np.array(['test', 'test', 'test1']), errs.index]

In [21]:
errs

ATVI        EA      TTWO    UBSFY
test  MAE               1.35850   1.34270   3.54050  0.25780
      RMSE              1.88670   1.89560   4.50690  0.33760
test1 Close Price Mean 83.64834 139.18954 170.87719 12.83658

In [22]:
with open(save_path / "metrics_err.tex", "w") as f:
    f.write(
        errs
        .to_latex(caption="Error metrics for chosen companies")
    )


C:\Users\kuba1\AppData\Local\Temp\ipykernel_9000\2401312644.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  errs


In [10]:
errs.T

,MAE,RMSE,Close Price Mean
ATVI,1.35850,1.88670,83.64834
EA,1.34270,1.89560,139.18954
TTWO,3.54050,4.50690,170.87719
UBSFY,0.25780,0.33760,12.83658


In [49]:
(errs.T['MAE'] / errs.T['Close Price Mean']) * 100

ATVI     1.624061
EA       0.964656
TTWO     2.071956
UBSFY    2.008324
dtype: float64

In [21]:
hparams = {
    "top_cut_off": np.arange(0, 4, 0.2),
    "down_cut_off": np.arange(0, 4, 0.2),
    "if_short": [True, False] + 18 * [None],
}

In [22]:
hparams_df = pd.DataFrame(hparams)

In [24]:
hparams_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
top_cut_off,0.0,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0,3.2,3.4,3.6,3.8
down_cut_off,0.0,0.2,0.4,0.6,0.8,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0,3.2,3.4,3.6,3.8
if_short,True,False,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [28]:
with open(save_path / "signals_hpars.tex", "w") as f:
    f.write(
        hparams_df
        .to_latex(caption="Signal hyper parameters")
    )


C:\Users\kuba1\AppData\Local\Temp\ipykernel_10816\2764230600.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  hparams_df
